# Malware Detection in Network Traffic Data

## Imports and spark setup

In [1]:
import pyspark
from tqdm import tqdm
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SQLContext, SparkSession

spark = SparkSession.builder.appName('Malware').getOrCreate()
#sc = spark.sparkContext
#sqlsc = SQLContext(spark)

import os
os.environ['HADOOP_HOME'] = 'C:/dummy/hadoop_home'

In [2]:
print(spark.sparkContext.getConf().getAll())

[('spark.executor.id', 'driver'), ('spark.app.submitTime', '1702383918978'), ('spark.driver.port', '55827'), ('spark.app.id', 'local-1702383919644'), ('spark.app.startTime', '1702383919042'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.refl

## Presentation

In [3]:
data = r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv"

In [4]:
df = spark.read.option("escape","\"").option("delimiter", "|").csv(data, header='true', inferSchema='true')

In [5]:
rows, cols = df.count(), len(df.columns)
rows, cols

(1008748, 23)

In [6]:
df.printSchema()

root
 |-- ts: double (nullable = true)
 |-- uid: string (nullable = true)
 |-- id.orig_h: string (nullable = true)
 |-- id.orig_p: double (nullable = true)
 |-- id.resp_h: string (nullable = true)
 |-- id.resp_p: double (nullable = true)
 |-- proto: string (nullable = true)
 |-- service: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- orig_bytes: string (nullable = true)
 |-- resp_bytes: string (nullable = true)
 |-- conn_state: string (nullable = true)
 |-- local_orig: string (nullable = true)
 |-- local_resp: string (nullable = true)
 |-- missed_bytes: double (nullable = true)
 |-- history: string (nullable = true)
 |-- orig_pkts: double (nullable = true)
 |-- orig_ip_bytes: double (nullable = true)
 |-- resp_pkts: double (nullable = true)
 |-- resp_ip_bytes: double (nullable = true)
 |-- tunnel_parents: string (nullable = true)
 |-- label: string (nullable = true)
 |-- detailed-label: string (nullable = true)



In [7]:
df.show(5)

+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|                 ts|               uid|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|
+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|1.525879831015811E9|CUmrqr4svHuSXJy5z7|192.168.100.103|  51524.0| 65.127.233.163|     23.0|  tcp|      -|2.999051|         0|         0|     

## Dataset concatenation

In [8]:
# List of file paths
file_paths = [
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-3-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-9-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-20-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-21-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-34-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-35-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-42-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-44-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-48-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-60-1conn.log.labeled.csv",
]

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Read the first file into a DataFrame (to get the schema)
df = spark.read.option("escape", "\"").option("delimiter", "|").csv(file_paths[0], header=True, inferSchema=True)

# Use tqdm for progress bar
for file_path in tqdm(file_paths[1:], desc="Reading files", unit="file"):
    df_temp = spark.read.option("escape", "\"").option("delimiter", "|").csv(file_path, header=True, inferSchema=True)
    df = df.union(df_temp)

# Show the first 5 rows and print the schema
df.show(5)
df.printSchema()


Reading files: 100%|██████████| 10/10 [00:22<00:00,  2.22s/file]


+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|                 ts|               uid|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|
+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|1.525879831015811E9|CUmrqr4svHuSXJy5z7|192.168.100.103|  51524.0| 65.127.233.163|     23.0|  tcp|      -|2.999051|         0|         0|     

## Data understanding

### dimension and content analysis

In [9]:
#replacing dots with underscores in column names to avoid errors
df = df.toDF(*(c.replace('.', '_') for c in df.columns))

In [10]:
rows, cols = df.count(), len(df.columns)
print(f'Dimension of the Dataframe is: {(rows,cols)}')

Dimension of the Dataframe is: (25000600, 23)


In [11]:
num_cols = [item[0] for item in df.dtypes if item[1] != 'string']
print('Le colonne numeriche sono')
print(num_cols)

Le colonne numeriche sono
['ts', 'id_orig_p', 'id_resp_p', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']


In [12]:
non_num_cols = [item[0] for item in df.dtypes if item[1] == 'string']
print('Le colonne non numeriche sono')
print(non_num_cols)

Le colonne non numeriche sono
['uid', 'id_orig_h', 'id_resp_h', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'history', 'tunnel_parents', 'label', 'detailed-label']


In [13]:
from pyspark.sql.functions import isnan, when, count, col, isnull
missing = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+---+---+---------+---------+---------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
| ts|uid|id_orig_h|id_orig_p|id_resp_h|id_resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|label|detailed-label|
+---+---+---------+---------+---------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
|  0|  0|        0|        0|        0|        0|    0|      0|       0|         0|         0|         0|         0|         0|           0|      0|        0|            0|        0|            0|             0|    0|       9173954|
+---+---+---------+---------+---------+---------+-----+-------+-----

### Check for missing labels for each file to understandand labelling consistency

In [14]:
from pyspark.sql.functions import isnan, when, count, col, isnull


def check_nan(spark, column_to_check="", label_only=False):
    # List of file numbers
    file_numbers = [1, 3, 9, 20, 21, 34, 35, 42, 44, 48, 60]
    
    for number in tqdm(file_numbers, desc="Processing files", unit="file"):
    
        file_path = r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-{}-1conn.log.labeled.csv".format(number)
        df = spark.read.option("escape", "\"").option("delimiter", "|").csv(file_path, header='true', inferSchema='true')
        df = df.toDF(*(c.replace('.', '_') for c in df.columns))

        print("", flush=True)
        # Check for missing values
        if label_only:
            missing = df.select([count(when(isnull("detailed-label"), "detailed-label")).alias("missing_count")])
            print("Missing values in file {}: ".format(number))
            missing.select("missing_count").show()

        elif column_to_check:
            missing = df.select([count(when(isnull(column_to_check), column_to_check)).alias("missing_count")])
            print("Missing values in file {}: ".format(number))
            missing.select("missing_count").show()

        elif column_to_check == "all":
            missing = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])
            print("Missing values in file {}: ".format(number))
            missing.show()

        else:
            print("Please enter a valid column name or enter True to check the label column only")

# Check for missing values in the label column only
check_nan(spark, label_only=True)


Processing files:   0%|          | 0/11 [00:00<?, ?file/s]


Missing values in file 1: 


Processing files:   9%|▉         | 1/11 [00:01<00:18,  1.87s/file]

+-------------+
|missing_count|
+-------------+
|            0|
+-------------+


Missing values in file 3: 


Processing files:  18%|█▊        | 2/11 [00:02<00:09,  1.07s/file]

...

Missing values in file 44: 
+-------------+
|missing_count|
+-------------+
|           15|
+-------------+


Missing values in file 48: 


Processing files:  91%|█████████ | 10/11 [00:31<00:03,  3.24s/file]

+-------------+
|missing_count|
+-------------+
|      3388871|
+-------------+


Missing values in file 60: 


Processing files: 100%|██████████| 11/11 [00:35<00:00,  3.23s/file]

+-------------+
|missing_count|
+-------------+
|      3578457|
+-------------+



### Check for distinct values and raw statistics

In [15]:
from pyspark.sql.functions import col, countDistinct

def count_distinct_values(df):
    result = {}
    columns = df.columns

    # Use tqdm to create a progress bar
    for column in tqdm(columns, desc="Counting Distinct Values", unit="column"):
        distinct_count = df.select(column).agg(countDistinct(column)).collect()[0][0]
        result[column] = distinct_count

    return result

distinct_counts = count_distinct_values(df)

# Print the result
for column, count in distinct_counts.items():
    print(f"Column '{column}' has {count} distinct values.")

Counting Distinct Values: 100%|██████████| 23/23 [05:47<00:00, 15.11s/column]

Column 'ts' has 25000600 distinct values.
Column 'uid' has 25000600 distinct values.
Column 'id_orig_h' has 21442 distinct values.
Column 'id_orig_p' has 65536 distinct values.
Column 'id_resp_h' has 11654577 distinct values.
Column 'id_resp_p' has 65435 distinct values.
Column 'proto' has 3 distinct values.
Column 'service' has 7 distinct values.
Column 'duration' has 1045931 distinct values.
Column 'orig_bytes' has 66149 distinct values.
Column 'resp_bytes' has 886 distinct values.
Column 'conn_state' has 13 distinct values.
Column 'local_orig' has 1 distinct values.
Column 'local_resp' has 1 distinct values.
Column 'missed_bytes' has 23 distinct values.
Column 'history' has 264 distinct values.
Column 'orig_pkts' has 207 distinct values.
Column 'orig_ip_bytes' has 1858 distinct values.
Column 'resp_pkts' has 129 distinct values.
Column 'resp_ip_bytes' has 1889 distinct values.
Column 'tunnel_parents' has 1 distinct values.
Column 'label' has 7 distinct values.
Column 'detailed-label

In [16]:
# Use the describe function to get statistical summary

summary = df.describe()
summary.show()


+-------+--------------------+------------------+------------+------------------+-----------+------------------+--------+--------+-----------------+--------------------+--------------------+----------+----------+----------+-----------------+--------+------------------+-----------------+--------------------+------------------+--------------+--------------------+--------------+
|summary|                  ts|               uid|   id_orig_h|         id_orig_p|  id_resp_h|         id_resp_p|   proto| service|         duration|          orig_bytes|          resp_bytes|conn_state|local_orig|local_resp|     missed_bytes| history|         orig_pkts|    orig_ip_bytes|           resp_pkts|     resp_ip_bytes|tunnel_parents|               label|detailed-label|
+-------+--------------------+------------------+------------+------------------+-----------+------------------+--------+--------+-----------------+--------------------+--------------------+----------+----------+----------+-----------------+-

In [17]:
num_summary = summary.select(*num_cols)
non_num_summary = summary.select(*non_num_cols)

print("Summary of numeric columns:")
num_summary.show()

print("Summary of non-numeric columns:")
non_num_summary.show()

Summary of numeric columns:
+--------------------+------------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+
|                  ts|         id_orig_p|         id_resp_p|     missed_bytes|         orig_pkts|    orig_ip_bytes|           resp_pkts|     resp_ip_bytes|
+--------------------+------------------+------------------+-----------------+------------------+-----------------+--------------------+------------------+
|            25000600|          25000600|          25000600|         25000600|          25000600|         25000600|            25000600|          25000600|
| 1.545444771111329E9| 39912.79891898594| 9714.519342175789|76.36013899666408|13.508834027983328|590.2281178451717|0.014572610257353824|1.7482263225682584|
|1.2003567012113284E7|16090.793857809633|22178.223936131817|381759.3150179955|17515.340014021454|687953.4074515603|  4.1902265695645475|  376.273102655407|
| 1.525879831015073E9|              